<a href="https://colab.research.google.com/github/Jerry086/SALSA/blob/Shiqing_audioset_embedding_checking/Different_sample_selection_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np

metadata = pd.read_csv('/content/drive/MyDrive/audioset/audio_metadata.csv')
embeddings = pd.read_csv('/content/drive/MyDrive/audioset/audio_embeddings.csv')

In [4]:
# Preprocessing the labels in metadata to ensure they are list objects and not strings
metadata['labels'] = metadata['labels'].apply(eval)

In [5]:
# Merge DataFrames on 'video_id'
merged_df = pd.merge(metadata, embeddings, on="video_id")
merged_df

,video_id,start_time_seconds,end_time_seconds,labels,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,...,feature_1270,feature_1271,feature_1272,feature_1273,feature_1274,feature_1275,feature_1276,feature_1277,feature_1278,feature_1279
0,wqoOX8K8DEU,30.0,40.0,"[396, 397]",89,255,19,54,240,199,...,101,152,255,0,194,190,0,91,247,251
1,wqH6Sj_h948,120.0,130.0,"[0, 441, 443]",89,74,221,113,99,254,...,98,38,255,0,255,255,124,29,0,255
2,wq1098my4zA,130.0,140.0,"[27, 137, 271]",224,124,142,123,113,144,...,72,116,62,197,89,51,212,44,112,177
3,wqR7LHho-WE,10.0,20.0,"[0, 22, 25]",147,207,173,147,216,98,...,86,81,91,123,192,51,255,255,101,0
4,wq6Me-UUbSc,360.0,370.0,[413],70,90,168,90,220,90,...,91,184,83,32,131,167,167,111,180,101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21777,2w6tV5kDGWo,240.0,250.0,[178],155,99,141,33,81,109,...,217,196,195,104,81,0,47,194,56,196
21778,2wZCoeq9Ppc,80.0,90.0,"[137, 138, 185, 195, 196, 198, 268]",102,255,136,0,0,95,...,80,0,233,146,255,200,0,0,0,96
21779,2wajg-UP-Gs,0.0,10.0,[459],132,72,60,190,75,6,...,255,156,156,35,255,0,0,255,255,0
21780,lZavPVn7O4Q,180.0,190.0,"[137, 258, 260, 273]",224,112,85,133,151,193,...,127,207,66,83,106,204,154,74,145,62


In [6]:
# Separate the embeddings (features) and labels for training
labels = merged_df['labels'].values
features = merged_df.drop(columns=['video_id', 'start_time_seconds', 'end_time_seconds', 'labels']).values

In [20]:
def check_label_overlap(reference_label, labels_results):
  cnt = 0
  # Flatten labels_results if it's nested in an extra layer
  if len(labels_results) == 1 and isinstance(labels_results[0], (list, np.ndarray)):
    labels_results = labels_results[0]
  reference_label_set = set(reference_label)
  for neighbor_labels in labels_results:
    neighbor_label_set = set(neighbor_labels)
    # Check for any overlap between the reference and neighbor labels
    overlap = not reference_label_set.isdisjoint(neighbor_label_set)
    if overlap:
      cnt += 1
  return cnt / 10

In [24]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KDTree

def select_samples_and_cal_precision():
  # Randomly select 5000 indices from the range of features' length
  selected_indices = np.random.choice(len(features), 5000, replace=False)

  # Select those indices from features and labels
  selected_features = features[selected_indices]
  selected_labels = labels[selected_indices]

  # randomly choose 1 sample from these 5000 to act as the reference
  reference_index = np.random.choice(len(selected_features), 1)[0]
  reference_feature = selected_features[reference_index].reshape(1, -1)
  reference_label = selected_labels[reference_index]

  # The rest of the samples will act as the "pool" for comparison
  pool_features = np.delete(selected_features, reference_index, axis=0)
  pool_labels = np.delete(selected_labels, reference_index)

  # Build a KDTree for the training embeddings
  kd_tree = KDTree(pool_features, leaf_size=40, metric='euclidean')

  # query the KDTree
  distances, indices = kd_tree.query(reference_feature, k=10)

  # get labels of results
  labels_results = pool_labels[indices]

  precision = check_label_overlap(reference_label, labels_results)

  return precision


In [25]:
# Run the tests for 2000 times:
# Divide the 2000 tests into 10 batches of 200 tests each
from scipy import stats

batch_means = []
for batch in range(10):
  precision = []
  for test in range(200):
    precision.append(select_samples_and_cal_precision())
  batch_mean = np.average(precision)
  batch_means.append(batch_mean)

print(batch_means)

# Perform one-way ANOVA
f_value, p_value = stats.f_oneway(*[batch_means[i::10] for i in range(10)])

print(f"F-Value: {f_value}, P-Value: {p_value}")

[0.242, 0.22649999999999998, 0.20900000000000002, 0.218, 0.276, 0.2525, 0.2575, 0.228, 0.2565, 0.2755]
F-Value: nan, P-Value: nan


/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:4141: DegenerateDataWarning: all input arrays have length 1.  f_oneway requires that at least one input has length greater than 1.
  warnings.warn(stats.DegenerateDataWarning(msg))
